In [ ]:
'''
Project Euler Problem 068: Magic 5-gon Ring
'''

In [ ]:
import sympy as sp
from itertools import permutations

In [ ]:
def pick_outer(n, highest, sum, start=1):
    # print(start, n, sum)

    selection = []
    if n == 1:
        selection.append([] if sum > highest else [sum])
    for value in range(start, min((sum-(n-1)*(n)//2)//n+1, highest)):
        if n > 1:
            subsets = pick_outer(n-1, highest, sum-value, value+1)
            for subset in subsets:
                if len(subset) == n-1:
                    selection.append([value]+subset)

    return selection


def complement(xlist, n):
    return [x for x in range(1, n+1) if x not in xlist]

In [ ]:
outer = sp.IndexedBase('o')
inner = sp.IndexedBase('i')
i = sp.symbols('i', cls=sp.Idx)
S = sp.symbols('S')

n = 5
nleg = 3

equations = [sp.Eq(outer[i]+sum([inner[(i+j)%n] for j in range(1, nleg)]), S) for i in range(n)]
outers = [outer[i] for i in range(n)]
inners = [inner[i] for i in range(n)]
solution = sp.solve(equations, inners)

In [ ]:
f_inners = [sp.lambdify(outers, solution[k]) for k in solution]
solutions = []

for outer_sum in range(1, n*(2*n+1)+1):
    inner_sum = n*(2*n+1) - outer_sum
    leg_sum = (outer_sum + (nleg-1)*inner_sum)
    # print(outer_sum, inner_sum, leg_sum)

    if leg_sum % n == 0:
        leg_sum //= n
        for pick in pick_outer(n, 2*n, outer_sum):
            for perm in permutations(pick[1:]):
                perm = [pick[0]]+list(perm)
                if sorted(pick_inner := [int(f(*perm).subs(S, leg_sum)) for f in f_inners]) == complement(perm, 2*n):
                    sol = [[[perm[i%n]]+[pick_inner[(i+j)%n] for j in range(1, nleg)] for i in range(n)]]
                    solutions += sol

legstrings = [''] * len(solutions)
for i, sol in enumerate(solutions):
    for leg in sol:
        legstrings[i] += ''.join(map(str, leg))

print(f'There are {len(solutions)} solutions:')
print(solutions)

lengths = set([len(s) for s in legstrings])

for length in lengths:
    legvalues = [int(ls) for ls in legstrings if len(ls) == length]
    print(f'The maximum string with {length} digits has a value {max(legvalues)} ')
